In [3]:
import os
import json

def find_best_config_str(exp_path, mode='retrieval', score_by='MAP'):
    l = []
    # find al files called log.txt, recursively
    for root, dirs, files in os.walk(exp_path):
        for file in files:
            if file.endswith('log.txt'):
                # open file, and for each line test if it contains the string "result for verification run - "
                with open(os.path.join(root, file)) as f:
                    for line in f:
                        if f'result for {mode} run' in line:
                            score = float(line.split(f'{score_by}: ')[1].split(' ')[0])
                            # now, get string between the brackets { and }
                            configs = '{' + line.split('{')[1].split('}')[0] + '}'
                            # replace single quotes with double quotes
                            configs = configs.replace("'", '"')
                            # convert each boolean value to string
                            configs = configs.replace(': True', ': "True"')
                            configs = configs.replace(': False', ': "False"')
                            # convert to json
                            config = json.loads(configs)
                            l.append((score, config))

    # sort the list by the first element, which is the score
    l.sort(key=lambda x: x[0], reverse=True)
    return l

In [4]:
def find_best_config_fp(exp_path, mode='retrieval', score_by='MAP'):
    l = []
    # find al files called log.txt, recursively
    for root, dirs, files in os.walk(exp_path):
        for file in files:
            if file.endswith('log.txt'):
                # open file, and for each line test if it contains the string "result for verification run - "
                with open(os.path.join(root, file)) as f:
                    for line in f:
                        if f'result for {mode} run' in line:
                            score = float(line.split(f'{score_by}: ')[1].split(' ')[0])
                            config = json.load(open(os.path.join(f.name, '..', '..', 'config.json')))
                            l.append((score, config))

    # sort the list by the first element, which is the score
    l.sort(key=lambda x: x[0], reverse=True)
    return l

exp_path = 'data-out/experiments/test-all/'

mode = 'verification'
score_by='F1'
configs = find_best_config_fp(exp_path, mode, score_by)

In [5]:
exp_path = 'data-out/experiments/test-dev'

# # print the top 10 results
# mode = 'retrieval'
# score_by='MAP'
# configs = find_best_config(exp_path, mode, score_by)
# for i, (f1, config) in enumerate(configs[:2]):
#     print(f'{i}. {score_by} score: {f1:.4f}, config: {config}')

# print the top 10 results
mode = 'verification'
score_by='Strict-F1'
configs = find_best_config_str(exp_path, mode, score_by)
for i, (score, config) in enumerate(configs[:]):
    print(f'{i}. {score_by} score: {score:.4f}, config: {config}')


0. Strict-F1 score: 0.9230, config: {'blind_run': 'False', 'split': 'dev', 'add_author_name': 'False', 'add_author_bio': 'False', 'retriever_k': 5, 'out_dir': './data-out/experiments/test-dev/retr-OPENAI/nopre-noname-nobio', 'preprocess': 'False', 'retriever_label': 'OPENAI', 'verifier_label': 'OPENAI', 'normalize_scores': 'False', 'scale': 'False', 'ignore_nei': 'True', 'fingerprint': 'nonorm-noscale-ignore_nei'}
1. Strict-F1 score: 0.9230, config: {'blind_run': 'False', 'split': 'dev', 'add_author_name': 'False', 'add_author_bio': 'False', 'retriever_k': 5, 'out_dir': './data-out/experiments/test-dev/retr-OPENAI/nopre-noname-nobio', 'preprocess': 'False', 'retriever_label': 'OPENAI', 'verifier_label': 'OPENAI', 'normalize_scores': 'False', 'scale': 'False', 'ignore_nei': 'False', 'fingerprint': 'nonorm-noscale-noignore_nei'}
2. Strict-F1 score: 0.9230, config: {'blind_run': 'False', 'split': 'dev', 'add_author_name': 'False', 'add_author_bio': 'False', 'retriever_k': 5, 'out_dir': '.

In [15]:
configs = find_best_config_str(exp_path, mode, score_by)
for i, (score, config) in enumerate(configs[:]):
    if config['retriever_label'] == 'TERRIER':
        print(f'{mode} - {i}. {score_by} score: {score:.4f}, config: {config["out_dir"]}/{config["fingerprint"]} - name:{config["add_author_name"]} bio:{config["add_author_bio"]} {config["verifier_label"]}')

verification - 6. F1 score: 0.8821, config: ./data-out/experiments/test-dev/retr-TERRIER/pre-name-nobio/nonorm-noscale-ignore_nei - name:True bio:False OPENAI
verification - 7. F1 score: 0.8821, config: ./data-out/experiments/test-dev/retr-TERRIER/pre-name-nobio/nonorm-noscale-noignore_nei - name:True bio:False OPENAI
verification - 8. F1 score: 0.8821, config: ./data-out/experiments/test-dev/retr-TERRIER/pre-name-nobio/nonorm-scale-ignore_nei - name:True bio:False OPENAI
verification - 9. F1 score: 0.8821, config: ./data-out/experiments/test-dev/retr-TERRIER/pre-name-nobio/nonorm-scale-noignore_nei - name:True bio:False OPENAI
verification - 10. F1 score: 0.8821, config: ./data-out/experiments/test-dev/retr-TERRIER/pre-name-nobio/norm-noscale-ignore_nei - name:True bio:False OPENAI
verification - 11. F1 score: 0.8821, config: ./data-out/experiments/test-dev/retr-TERRIER/pre-name-nobio/norm-noscale-noignore_nei - name:True bio:False OPENAI
verification - 12. F1 score: 0.8306, config: .

In [6]:
# find best config for verification overall
mode = 'verification'
score_by='F1'
configs = find_best_config_fp(f'./data-out/experiments/test-dev', mode, score_by)
for i, (f1, config) in enumerate(configs[0:4]):
    print(f'{mode} - {i+1}. {score_by} score: {f1:.4f}, config: {config["out_dir"]}/{config["fingerprint"]} - {config["verifier_label"]}')

verification - 1. F1 score: 0.9230, config: ./data-out/experiments/test-dev/retr-OPENAI/nopre-noname-nobio/nonorm-noscale-ignore_nei - OPENAI
verification - 2. F1 score: 0.9230, config: ./data-out/experiments/test-dev/retr-OPENAI/nopre-noname-nobio/nonorm-noscale-noignore_nei - OPENAI
verification - 3. F1 score: 0.9230, config: ./data-out/experiments/test-dev/retr-OPENAI/nopre-noname-nobio/nonorm-scale-ignore_nei - OPENAI
verification - 4. F1 score: 0.9230, config: ./data-out/experiments/test-dev/retr-OPENAI/nopre-noname-nobio/norm-noscale-ignore_nei - OPENAI


In [7]:

try:
    no_ext_data = []
    ext_data = []
    for item in configs:
        config = item[1]
        if config['add_author_name'] or config['add_author_bio']:
            ext_data.append(item)
        else:
            no_ext_data.append(item)
    
    no_ext_data = sorted(no_ext_data, key=lambda x: x[0], reverse=True)
    ext_data = sorted(ext_data, key=lambda x: x[0], reverse=True)

    for i, (f1, config) in enumerate(no_ext_data):
        print(f'{mode} no ext data - {i+1}. {score_by} score: {f1:.4f}, config: {config}')
except:
    pass

verification no ext data - 1. F1 score: 0.9230, config: {'blind_run': False, 'split': 'dev', 'add_author_name': False, 'add_author_bio': False, 'retriever_k': 5, 'out_dir': './data-out/experiments/test-dev/retr-OPENAI/nopre-noname-nobio', 'preprocess': False, 'retriever_label': 'OPENAI', 'verifier_label': 'OPENAI', 'normalize_scores': False, 'scale': False, 'ignore_nei': True, 'fingerprint': 'nonorm-noscale-ignore_nei'}
verification no ext data - 2. F1 score: 0.9230, config: {'blind_run': False, 'split': 'dev', 'add_author_name': False, 'add_author_bio': False, 'retriever_k': 5, 'out_dir': './data-out/experiments/test-dev/retr-OPENAI/nopre-noname-nobio', 'preprocess': False, 'retriever_label': 'OPENAI', 'verifier_label': 'OPENAI', 'normalize_scores': False, 'scale': False, 'ignore_nei': False, 'fingerprint': 'nonorm-noscale-noignore_nei'}
verification no ext data - 3. F1 score: 0.9230, config: {'blind_run': False, 'split': 'dev', 'add_author_name': False, 'add_author_bio': False, 'retr

In [8]:

try:
    no_ext_data = []
    ext_data = []
    for item in configs:
        config = item[1]
        if config['add_author_name'] or config['add_author_bio']:
            ext_data.append(item)
        else:
            no_ext_data.append(item)
    
    no_ext_data = sorted(no_ext_data, key=lambda x: x[0], reverse=True)
    ext_data = sorted(ext_data, key=lambda x: x[0], reverse=True)


    for i, (f1, config) in enumerate(ext_data):
        print(f'{mode} using ext data - {i+1}. {score_by} score: {f1:.4f}, config: {config}')    
except:
    pass

verification using ext data - 1. F1 score: 0.8821, config: {'blind_run': False, 'split': 'dev', 'add_author_name': True, 'add_author_bio': False, 'retriever_k': 5, 'out_dir': './data-out/experiments/test-dev/retr-TERRIER/pre-name-nobio', 'preprocess': True, 'retriever_label': 'TERRIER', 'verifier_label': 'OPENAI', 'normalize_scores': False, 'scale': False, 'ignore_nei': True, 'fingerprint': 'nonorm-noscale-ignore_nei'}
verification using ext data - 2. F1 score: 0.8821, config: {'blind_run': False, 'split': 'dev', 'add_author_name': True, 'add_author_bio': False, 'retriever_k': 5, 'out_dir': './data-out/experiments/test-dev/retr-TERRIER/pre-name-nobio', 'preprocess': True, 'retriever_label': 'TERRIER', 'verifier_label': 'OPENAI', 'normalize_scores': False, 'scale': False, 'ignore_nei': False, 'fingerprint': 'nonorm-noscale-noignore_nei'}
verification using ext data - 3. F1 score: 0.8821, config: {'blind_run': False, 'split': 'dev', 'add_author_name': True, 'add_author_bio': False, 'retr